# First part - let us fix the problems with our database

In [2]:
import sqlite3

con = sqlite3.connect('../db/vivino.db')
cursor = con.cursor()


##### This is to fix the wineries table. We are going to essentially transfer the data that is the winery name from the vintage names, by subtracting strings. WineryName = VintageName - WineName - Year

In [2]:
try:
    cursor.execute(
        """
        ALTER TABLE vintages
        ADD COLUMN wineries TEXT;
    """
    )
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        UPDATE vintages
        SET wineries = SUBSTR(name, 1, LENGTH(name) - 5)
    """
    )
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        ALTER TABLE wines
        ADD COLUMN wineries TEXT;
    """
    )
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        UPDATE wines
        SET wineries = (SELECT wineries FROM vintages WHERE vintages.wine_id = wines.id)
    """
    )
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        UPDATE wines
        SET wineries = CASE
            WHEN instr(wineries, name) = 0 THEN wineries
            ELSE substr(wineries, 1, instr(wineries, name) - 1) || 
                 substr(wineries, instr(wineries, name) + length(name))
        END
        """
    )
    con.commit()

    print("Update successful!")
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

Error: duplicate column name: wineries
Update successful!
Update successful!
Update successful!
Update successful!
Update successful!


##### Now we have to clear out wineries, and insert the new data

In [3]:
try:
    cursor.execute(
        """
        DELETE FROM wineries        
        """
    )
    con.commit()
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        INSERT OR IGNORE INTO wineries (id, name)
        SELECT DISTINCT winery_id, wineries 
        FROM wines
        """
    )
    con.commit()
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        ALTER TABLE wines
        DROP COLUMN wineries
        """
    )
    con.commit()
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

try:
    cursor.execute(
        """
        ALTER TABLE vintages
        DROP COLUMN wineries
        """
    )
    con.commit()
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

##### Trim the spacebars

In [3]:
try:
    cursor.execute(
        """
        UPDATE wineries
        SET name = TRIM(name)
    """
    )
    con.commit()

    print("Update successful!")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

Update successful!


##### Check UNIQUE Winery IDs to make sure we got them all


In [3]:
winery_id_from_wines = cursor.execute(
    """
    SELECT DISTINCT winery_id
    FROM wines
    """
).fetchall()

winery_id_from_wineries = cursor.execute(
    """
    SELECT id
    FROM wineries
    """
).fetchall()

print("Length of winery_id_from_wines:", len(winery_id_from_wines))
print("Length of winery_id_from_wineries:", len(winery_id_from_wineries))
    

Length of winery_id_from_wines: 658
Length of winery_id_from_wineries: 658


# Now that we have fixed the DataBase, we will create some views to analyze data.

##### Create the View "Country Priority", to give us the countries and the user population count that uses the website

In [6]:
try:
    cursor.execute(
        """
        CREATE VIEW country_priority AS
        SELECT c.name, c.users_count, c.regions_count, c.wines_count, c.wineries_count
        FROM countries as c
        ORDER BY c.users_count DESC
    """
    )
    con.commit()
    print("view created")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

view created


##### Now the Top Five Wines per Grape Type view.

In [7]:
try:
    cursor.execute(
        """
        CREATE VIEW top_five_wines_per_grape AS
        WITH top_grapes AS (
            SELECT g.id AS id, g.name AS grape, g.wines_count AS wines_count
            FROM grapes AS g
            ORDER BY g.wines_count DESC
        ),
        ranked_wines AS (
            SELECT w.name AS wine, w.winery_id AS winery_id, w.grape_id, 
                   ROW_NUMBER() OVER(PARTITION BY w.grape_id ORDER BY w.ratings_average DESC) as rn
            FROM wines AS w
        )
        SELECT tg.grape, rw.wine, wi.name AS winery
        FROM top_grapes AS tg
        JOIN ranked_wines AS rw ON tg.id = rw.grape_id
        JOIN wineries AS wi ON rw.winery_id = wi.id
        WHERE rw.rn <= 5
        ORDER BY tg.wines_count DESC, rw.rn
        """
    )
    con.commit()
    print("view created")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

view created


##### These are in case I need to make any changes, these delete the views so i can create new ones

In [4]:
cursor.execute(
    """
    DROP VIEW IF EXISTS country_priority
    """
)
con.commit()

In [5]:
cursor.execute(
    """
    DROP VIEW IF EXISTS top_five_wines_per_grape
    """
)
con.commit()